In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from math import log
import math

# Data Preperation

## Used DF

In [2]:
user_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\Top 20 Cryptocurrencies and Related TW Accounts.csv", encoding="ISO-8859-1")
user_df

,Coin,ID,Owner/Influence,Name,Screen_name,Status,Why did I choose this person?
0,Ethereum,ethereum,Vitalik Buterin,vitalik.eth,VitalikButerin,co-founder,he is also known in crypto world
1,BNB,binancecoin,Changpeng Zhao,CZ ??Binance,cz_binance,CEO,NaN
2,XRP,ripple,Brad Garlinghouse,Brad Garlinghouse,bgarlinghouse,CEO,NaN
3,ADA(Cardano),cardano,Charles Hoskinson,Charles Hoskinson,IOHK_Charles,CEO/co-founder,NaN
4,DOGE,dogecoin,Elon Musk_doge,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
5,MATIC(Polygon),matic-network,Sandeep Nailwal,Sandeep | Polygon ????3??,sandeepnailwal,co-founder,he seems more active than other co-founders
6,DOT(Polkadot),polkadot,WEB3 Foundation,Web3 Foundation,Web3foundation,foundation,there is no account of CEO or sth
7,HEX,hex,Richard Heart,Richard Heart,RichardHeartWin,CEO,NaN
8,SHIB,shiba-inu,Elon Musk_shiba,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
9,SOL,solana,Solana,Solana,solana,company,there is no account of CEO or sth


## Min-Max Day Finders

In [3]:
def last_day_finder(tweet_df):
    max_index = len(tweet_df)-1
    user = tweet_df["Screen_name"][0]
    last_day = tweet_df["Date"][0]
    for x in tweet_df.index:
        if(x != max_index):
            if(tweet_df["Screen_name"][x] != tweet_df["Screen_name"][x+1]):
                if(tweet_df["Date"][x+1] > last_day):
                    last_day = tweet_df["Date"][x+1]
                    user = tweet_df["Screen_name"][x+1]            
        elif(x == max_index):
            if(tweet_df["Date"][x] > last_day):
                last_day = tweet_df["Date"][x]
                user = tweet_df["Screen_name"][x]
    
    print("Last day of data is ", last_day, "and user who has the last tweet is", user)
    return last_day

In [4]:
#It finds the common oldest day between all users.
def common_oldest_day_finder(tweet_df):
    max_index = len(tweet_df)-1
    most_active_user = ""
    max_day = ""
    for x in tweet_df.index:
        if(x != max_index):
            if(tweet_df["Screen_name"][x] != tweet_df["Screen_name"][x+1]):
                if(tweet_df["Date"][x] > max_day):
                    max_day = tweet_df["Date"][x]
                    most_active_user = tweet_df["Screen_name"][x]
        elif(x == max_index):
            if(tweet_df["Date"][x] > max_day):
                max_day = tweet_df["Date"][x]
                most_active_user = tweet_df["Screen_name"][x]
    print("Oldest common day is", max_day, "and Most active user is", most_active_user)

In [5]:
#this method is being used in LR with different time intervals
#add oldest price day
def oldest_day_finder(df):
    oldest_price_day = "2022-06-15" #change this day according to price date interval
    oldest_price_day = datetime.strptime(oldest_price_day, '%Y-%m-%d')
    oldest_price_day = oldest_price_day.date()
    day = df["Date"].iloc[-1].split(" ")[0]
    day = datetime.strptime(day, '%Y-%m-%d')
    day = day + relativedelta(days = 1)
    day = day.date()
    if(oldest_price_day >= day):
        day = oldest_price_day
    
    return day.strftime('%Y-%m-%d')

In [6]:
tweet_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\Input Data\tweets_all.csv", encoding="ISO-8859-1")
tweet_df.drop("Unnamed: 0", inplace=True, axis=1)
common_oldest_day_finder(tweet_df)
last_day_finder(tweet_df)

Oldest common day is 2022-10-09 13:30:21 and Most active user is elonmusk
Last day of data is  2022-12-31 23:41:41 and user who has the last tweet is VitalikButerin


'2022-12-31 23:41:41'

In [7]:
#common day date interval
#check dates according to collection interval
#add 1 to max_day to ensure getting all tweets of most active user at specified day.
oldest_day = "2022-10-10"
last_day = "2022-12-31" #last day will be used for all coins
common_day_idx = pd.date_range(oldest_day, last_day, freq ='D')

## Seperator

In [8]:
def separator(index ,tweet_df):
    result = tweet_df
    result.insert(index, "Time", "", True)
    
    for x in result.index:
        date = result.loc[x]["Date"]
        result["Date"][x] = date.split(" ")[0]
        result["Time"][x] = date.split(" ")[1]
    return result

## Price Data

In [9]:
price_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\Input Data\price_data_v2.csv")
price_df.drop("Unnamed: 0", inplace=True, axis=1)
price_df.rename(columns={"date_time": "Date"}, inplace=True)
price_df = separator(4, price_df)
price_df.drop("Time", inplace=True, axis=1)

<ipython-input-8-0383d5a971fa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Date"][x] = date.split(" ")[0]
<ipython-input-8-0383d5a971fa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Time"][x] = date.split(" ")[1]


## Tweet Data

In [10]:
tweet_df.reset_index(inplace = True, drop = True)
tweet_df = separator(6, tweet_df)

<ipython-input-8-0383d5a971fa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Date"][x] = date.split(" ")[0]
<ipython-input-8-0383d5a971fa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Time"][x] = date.split(" ")[1]


In [11]:
def groupby(df):
    tweet_counts = df.sort_values(["Related_coin", "Date"], ascending=True).groupby(["Related_coin", "Date"], sort=False)["Tweet_id"].count()
    tw_count_df = tweet_counts.to_frame()
    tw_count_df.rename(columns={"Tweet_id": "Num_Tweets"}, inplace=True)
    tw_count_df = tw_count_df.reset_index(level=[0])
    return tw_count_df

In [12]:
#to fill missing dates for one coin with it's specified date interval
def one_filler(df, idx):
    coin_id = df["Related_coin"][0]
    df.index=pd.to_datetime(df.index)
    result = df.reindex(idx)
    result =  result.fillna(value={'Related_coin':coin_id,'Num_Tweets':0})
    result["LN"] = np.where(result['Num_Tweets']>0, np.log(result['Num_Tweets']), 0)
    return result

In [13]:
#to fill missing dates for all coins with same date interval
def all_filler(user_df ,tw_count_df, idx):
    result_df = pd.DataFrame()
    for x in user_df.index:
        coin_id = user_df["ID"][x]
        partial_df = tw_count_df[tw_count_df["Related_coin"] == coin_id]
        partial_res = one_filler(partial_df, idx)
        result_df = result_df.append(partial_res)
    return result_df

# Linear Regression

In [14]:
def variance(data, n, ddof=0): #ddof = 0 for population, =1 for sample variance 
    if(n != 0):
        mean = sum(data) / n
        return sum((x - mean) ** 2 for x in data) / (n - ddof)
    else:
        return 144

In [15]:
def std_err_calculator(data):
    n = len(data)
    var = variance(data, n)
    std_dev = math.sqrt(var)
    std_err = std_dev/math.sqrt(n)
    return std_err

## Common Day

In [16]:
def common_day_linearRegression(user_df, price_df, tweet_df, common_day_idx):
    columns = ["ID", "Screen_name", "R_sq", "Intercept", "coef", "t_stat"]
    data = []
    
    #data preparation
    #price
    price_df_v2 = price_df[price_df["Date"] >= oldest_day]
    price_df_v2.reset_index(inplace = True, drop = True)
    #tweets
    tweet_df_v2 = tweet_df[tweet_df["Date"] >= oldest_day]
    #groupby
    tw_count_df = groupby(tweet_df_v2)
    #fill
    tw_count_df = all_filler(user_df, tw_count_df, common_day_idx)
    
    for x in user_df.index:
        coin_id = user_df["ID"][x]
        user = user_df["Screen_name"][x]
        
        #independent
        var_x = price_df_v2[price_df_v2["ID"] == coin_id]["%change"].tolist()
        var_x = np.reshape(var_x, (-1,1))
        
        #dependent
        var_y = tw_count_df[tw_count_df["Related_coin"] == coin_id]["LN"].tolist()
        std_err = std_err_calculator(var_y) #standart error calculation
        var_y = np.reshape(var_y, [1, len(var_y)])[0]
        model = LinearRegression().fit(var_x, var_y)
        t_stat = model.coef_[0]/std_err
        
        
        data.append([coin_id, user, model.score(var_x, var_y), model.intercept_, model.coef_[0], t_stat])
    
    return pd.DataFrame(data, columns = columns)

In [17]:
def different_days_linearRegression(user_df, price_df, tweet_df, last_day):
    columns = ["ID", "Screen_name", "Oldest_day", "Last_day", "R_sq", "Intercept", "coef", "t_stat"]
    data = []
    
    for x in user_df.index:
        coin_id = user_df["ID"][x]
        user = user_df["Screen_name"][x]
        
        #slicing
            #tweet_df
        slice_tw = tweet_df.loc[tweet_df["Related_coin"] == coin_id]
        oldest_day = oldest_day_finder(slice_tw)
        slice_tw = slice_tw[(slice_tw["Date"] >= oldest_day) & (slice_tw["Date"] <= last_day)]
            #price_df
        slice_pr = price_df[(price_df["Date"] >= oldest_day) & (price_df["Date"] <= last_day)]
            #groupby
        tw_count_df = groupby(slice_tw)
            #fill
        tw_count_df = one_filler(tw_count_df, pd.date_range(oldest_day, last_day, freq ='D'))
        
        #Linear Regression
            #independent
        var_x = slice_pr[slice_pr["ID"] == coin_id]["%change"].tolist()
        var_x = np.reshape(var_x, (-1,1))
            #dependent
        var_y = tw_count_df[tw_count_df["Related_coin"] == coin_id]["LN"].tolist()
        std_err = std_err_calculator(var_y) #standart error calculation
        var_y = np.reshape(var_y, [1, len(var_y)])[0]
        model = LinearRegression().fit(var_x, var_y)
        t_stat = model.coef_[0]/std_err
        
        data.append([coin_id, user, oldest_day, last_day, model.score(var_x, var_y), model.intercept_, model.coef_[0], t_stat])
    
    return pd.DataFrame(data, columns = columns)

In [18]:
#to open/close 0 dividing warning for log
np.seterr(divide = 'ignore')
#np.seterr(divide = 'warn')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [19]:
common_day = common_day_linearRegression(user_df, price_df, tweet_df, common_day_idx)
common_day

,ID,Screen_name,R_sq,Intercept,coef,t_stat
0,ethereum,VitalikButerin,0.002178,1.777479,-0.994915,-9.833630
1,binancecoin,cz_binance,0.000002,2.631138,0.026505,0.320071
2,ripple,bgarlinghouse,0.005599,0.070032,-0.434783,-15.159795
3,cardano,IOHK_Charles,0.001750,1.973872,0.700760,9.493249
4,dogecoin,elonmusk,0.000286,3.178702,-0.130083,-2.044148
5,matic-network,sandeepnailwal,0.011330,2.715550,1.112570,13.890778
6,polkadot,Web3foundation,0.001642,0.171410,0.539595,10.057448
7,hex,RichardHeartWin,0.003965,1.932811,1.211164,11.702457
8,shiba-inu,elonmusk,0.000321,3.180324,0.239593,3.771426
9,solana,solana,0.000450,0.969637,-0.241508,-2.597869


In [20]:
different_day = different_days_linearRegression(user_df, price_df, tweet_df, last_day)
different_day

,ID,Screen_name,Oldest_day,Last_day,R_sq,Intercept,coef,t_stat
0,ethereum,VitalikButerin,2022-06-15,2022-12-31,0.002854,1.789065,1.070593,16.188707
1,binancecoin,cz_binance,2022-07-11,2022-12-31,0.000628,2.519911,-0.542614,-9.533346
2,ripple,bgarlinghouse,2022-06-15,2022-12-31,0.000285,0.063694,-0.094609,-5.626827
3,cardano,IOHK_Charles,2022-06-15,2022-12-31,0.034409,1.736437,-4.001478,-67.902947
4,dogecoin,elonmusk,2022-10-10,2022-12-31,0.000286,3.178702,-0.130083,-2.044148
5,matic-network,sandeepnailwal,2022-08-06,2022-12-31,0.006559,2.567009,0.961878,16.815428
6,polkadot,Web3foundation,2022-06-15,2022-12-31,0.003733,0.111749,-0.589300,-20.791496
7,hex,RichardHeartWin,2022-07-01,2022-12-31,0.000070,2.346383,0.160384,2.151237
8,shiba-inu,elonmusk,2022-10-10,2022-12-31,0.000321,3.180324,0.239593,3.771426
9,solana,solana,2022-06-15,2022-12-31,0.002994,0.909586,-0.743639,-12.528930


In [22]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\Regression Results\regression_results.xlsx", engine='xlsxwriter')

# Write each dataframe to a different worksheet.
common_day.to_excel(writer, sheet_name='common_day')
different_day.to_excel(writer, sheet_name='different_day')

# Close the Pandas Excel writer and output the Excel file.
writer.close()

## Different Days